In [38]:
'''
prompt:
Escreva código python que lê todos os arquivos PDF da pasta data/exams e converte eles para texto.
 A função parse_pdfs deve retornar uma lista de tuplas: o primeiro elemento da tupla é o nom do arquivo PDF e o segundo elemento é o texto de cada PDF. '''

'\nprompt:\nEscreva código python que lê todos os arquivos PDF da pasta data/exams e converte eles para texto.\n A função parse_pdfs deve retornar uma lista de tuplas: o primeiro elemento da tupla é o nom do arquivo PDF e o segundo elemento é o texto de cada PDF. '

In [39]:
!pip install pymupdf




[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\pedro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [40]:
import os
import fitz  # PyMuPDF

def parse_pdfs(pasta="data/exams"):
    resultados = []
    
    # Percorre todos os arquivos da pasta
    for nome_arquivo in os.listdir(pasta):
        if nome_arquivo.lower().endswith(".pdf"):
            caminho_pdf = os.path.join(pasta, nome_arquivo)
            try:
                with fitz.open(caminho_pdf) as doc:
                    texto = ""
                    for pagina in doc:
                        texto += pagina.get_text()
                resultados.append((nome_arquivo, texto))
            except Exception as e:
                print(f"Erro ao processar {nome_arquivo}: {e}")
    
    return resultados


In [41]:
pdf_textos = parse_pdfs()
for i, (pdf_filename, texto) in enumerate(pdf_textos):
    print(f"\n--- PDF {i+1} ---\n")
    print(pdf_filename, texto[:50])  # Mostra os primeiros 500 caracteres de cada PDF


pdf_texto = pdf_textos[0]
assert pdf_texto[0] == "oab-1.pdf"


--- PDF 1 ---

oab-1.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 2 ---

oab-10.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 3 ---

oab-100.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 4 ---

oab-101.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 5 ---

oab-102.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 6 ---

oab-103.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 7 ---

oab-104.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 8 ---

oab-105.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 9 ---

oab-106.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 10 ---

oab-107.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 11 ---

oab-108.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 12 ---

oab-109.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 13 ---

oab-11.pdf Questões comentadas OAB/

In [42]:
'''
prompt:
Escreva código python que recebe uma string de texto e retorna uma lista de dicionários. 
Cada elemento da lista deve conter as chaves "materia", "questão", "prova" e "tema", em que o valor é o texto entre a chave e \n". 
Exemplo: \nMatéria: ÉTICA PR OFISSIONAL\nQuestão:  001\nProva: XV\nTema:  INCOMP ATIBILID ADE E IMPEDIMENT O\n

Depois, o "enunciado" está entre o que vem depois do Tema e A). Leia A), B), C) e D) como entradas distintas no dicionário.

Em cada alternativa, há o texto "Resposta errada" ou "Resposta correta". Cada alternativa deve ter 3 sub-chaves: o texto, um booleano que diz se a resposta é correta, e a a justificativa que vem a seguir.

'''

'\nprompt:\nEscreva código python que recebe uma string de texto e retorna uma lista de dicionários. \nCada elemento da lista deve conter as chaves "materia", "questão", "prova" e "tema", em que o valor é o texto entre a chave e \n". \nExemplo: \nMatéria: ÉTICA PR OFISSIONAL\nQuestão:  001\nProva: XV\nTema:  INCOMP ATIBILID ADE E IMPEDIMENT O\n\n\nDepois, o "enunciado" está entre o que vem depois do Tema e A). Leia A), B), C) e D) como entradas distintas no dicionário.\n\nEm cada alternativa, há o texto "Resposta errada" ou "Resposta correta". Cada alternativa deve ter 3 sub-chaves: o texto, um booleano que diz se a resposta é correta, e a a justificativa que vem a seguir.\n\n'

In [43]:
import re

def parse_questoes(pdf_filename, texto):

    padrao_bloco = re.compile(r'\nMatéria: (.*?)\nQuestão: (.*?)\nProva: (.*?)\nTema: (.*?)\n', re.DOTALL)
    blocos = list(padrao_bloco.finditer(texto))
    questoes = []

    for i, bloco in enumerate(blocos):
        materia = bloco.group(1).strip()
        questao = bloco.group(2).strip()
        prova = bloco.group(3).strip()
        tema = bloco.group(4).strip()

        # Determinar os limites do bloco de questão
        inicio_enunciado = bloco.end()
        fim_bloco = blocos[i + 1].start() if i + 1 < len(blocos) else len(texto)
        conteudo_questao = texto[inicio_enunciado:fim_bloco]

        # Separar enunciado e alternativas
        alternativas = re.split(r'([A-D]\))', conteudo_questao)
        enunciado = alternativas[0].strip()
        alternativas_dict = {}

        for j in range(1, len(alternativas) - 1, 2):
            letra = alternativas[j].strip(')')
            corpo = alternativas[j + 1].strip()

            # Extrair texto, resposta e justificativa
            match = re.match(r'^(.*?)(Resposta (correta|errada))(.*)$', corpo, re.DOTALL)
            if match:
                texto_alt = match.group(1).strip()
                correta = match.group(3).strip().lower() == 'correta'
                justificativa = match.group(4).strip()
            else:
                texto_alt = corpo
                correta = None
                justificativa = ''

            alternativas_dict[letra] = {
                'texto': texto_alt,
                'correta': correta,
                'justificativa': justificativa
            }

        correct = None
        if alternativas_dict['A']['correta'] == True:
            correct = "A"
        if alternativas_dict['B']['correta'] == True:
            correct = "B"
        if alternativas_dict['C']['correta'] == True:
            correct = "C"
        if alternativas_dict['D']['correta'] == True:
            correct = "D"    

        questoes.append({
            'pdf_filename': pdf_filename,
            'materia': materia,
            'questao': questao,
            'prova': prova,
            'tema': tema,
            'enunciado': enunciado,
            'A': alternativas_dict['A']['texto'],
            'B': alternativas_dict['B']['texto'],
            'C': alternativas_dict['C']['texto'],
            'D': alternativas_dict['D']['texto'],
            'correct': correct
        })

    return questoes


### TESTES ####


pdf_file_texto_1 = pdf_textos[0]


questoes = parse_questoes(pdf_file_texto_1[0], pdf_file_texto_1[1])

assert len(questoes) == 10

questao = questoes[0]

assert questao['pdf_filename'] == 'oab-1.pdf'
assert questao['materia'] == 'ÉTICA PROFISSIONAL'
assert questao['prova'] == 'XV'
assert questao['questao'] == '001'

assert questao['enunciado'].startswith("Abelardo")

assert questao['A'] != None
assert questao['B'] != None
assert questao['C'] != None
assert questao['D'] != None

assert questao['correct'] == 'C'


In [44]:
'''
prompt:
Tenho uma lista de dicionários. Cada dicionário tem uma chave "prova", "materia", "questao" e "enunciado". 
Gere  código que produz um pandas dataframe salva em data/oab.csv
'''

'\nprompt:\nTenho uma lista de dicionários. Cada dicionário tem uma chave "prova", "materia", "questao" e "enunciado". \nGere  código que produz um pandas dataframe salva em data/oab.csv\n'

In [45]:
import pandas as pd

def salvar_provas(lista_de_questoes):
    # Cria o DataFrame
    df = pd.DataFrame(lista_de_questoes)

    # Garante a ordem das colunas
    colunas_desejadas = ["pdf_filename", "prova", "materia", "tema", "questao", "enunciado", "A", "B", "C", "D", "correct"]
    df = df[colunas_desejadas]


    return df

# Supondo que cada elemento de pdf_textos seja uma lista de dicionários
todas_questoes = []

pdf_count = 0
for pdf_texto in pdf_textos:
    questoes = parse_questoes(pdf_texto[0], pdf_texto[1])
    todas_questoes.extend(questoes)  # Adiciona todas as questões ao acumulador

    pdf_count += 1
    print('Processing...', pdf_count)


df = salvar_provas(todas_questoes)
df.to_csv("data/oab.csv", index=False, encoding="utf-8")
print(f"Arquivo salvo em data/oab.csv")

Processing... 1
Processing... 2
Processing... 3
Processing... 4
Processing... 5
Processing... 6
Processing... 7
Processing... 8
Processing... 9
Processing... 10
Processing... 11
Processing... 12
Processing... 13
Processing... 14
Processing... 15
Processing... 16
Processing... 17
Processing... 18
Processing... 19
Processing... 20
Processing... 21
Processing... 22
Processing... 23
Processing... 24
Processing... 25
Processing... 26
Processing... 27
Processing... 28
Processing... 29
Processing... 30
Processing... 31
Processing... 32
Processing... 33
Processing... 34
Processing... 35
Processing... 36
Processing... 37
Processing... 38
Processing... 39
Processing... 40
Processing... 41
Processing... 42
Processing... 43
Processing... 44
Processing... 45
Processing... 46
Processing... 47
Processing... 48
Processing... 49
Processing... 50
Processing... 51
Processing... 52
Processing... 53
Processing... 54
Processing... 55
Processing... 56
Processing... 57
Processing... 58
Processing... 59
Proces

In [46]:
df['materia'].value_counts().head(50)


materia
ÉTICA PROFISSIONAL        389
DIREITO CONSTITUCIONAL    295
DIREITO CIVIL             294
PROCESSO CIVIL            273
DIREITO PENAL             259
DIREITO DO TRABALHO       253
DIREITO ADMINISTRATIVO    252
PROCESSO PENAL            231
PROCESSO DO TRABALHO      207
DIREITO EMPRESARIAL       200
DIREITO TRIBUTÁRIO        200
DIREITOS HUMANOS          102
DIREITO DO CONSUMIDOR      85
DIREITO AMBIENTAL          84
ECA                        83
DIREITO INTERNACIONAL      81
FILOSOFIA                  68
DIREITO PREVIDENCIÁRIO     22
DIREITO ELEITORAL          22
DIREITO FINANCEIRO         22
Name: count, dtype: int64